<a href="https://colab.research.google.com/github/nunofernandes-plight/.NET-Library/blob/master/Build_Video_Generators_Vikit_ai_v0_2_Open_Source.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

Downloading the SDK

In [ ]:
!git clone https://github.com/Vikit-ai/sdk

In [ ]:
%cd sdk

# **Important : Get a VIKIT API Token at this adress https://vikit.ai/#/platform and set it up in the following cell before to run this colab**

In [ ]:
%%writefile .env.secrets.dev
VIKIT_API_TOKEN=Get you Vikit API Key here https://vikit.ai/#/platform

# Configuring the Colab with Vikit config, python packages, and ffmpeg to process the content

In [ ]:
%%writefile .env.config.dev
TESTMODE=true
ELEVEN_LABS_API_URI=https://api.elevenlabs.io/v1/text-to-speech/21m00Tcm4TlvDq8ikWAM
VIDEHO_EMAIL_CONTACT=clement@vikit.ai
NB_RETRIES_HTTP_CALLS=3
SUBTITLES_MIN_DURATION=7
STEPS=300
NUMBER_OF_SUBTITLES_PER_VIDEO_PROMPT=1
SUBTITLES_FILE_NAME = "subtitles.srt"
CLEANUP_TEMPFILES=false
SUB_AUDIO_FOR_SOUND_PREFIX = "sub_audio_for_subtitle_from"
PROMPT_MP3_FILE_NAME=prompt_upload.mp3
VIDEO_ENCODED_WITHOUT_MUSIC_FILE_PREFIX=final_video_without_music
INITIAL_AUDIO_FILE_NAME="upload.mp3"
VIDEO_LIST_FILE_NAME = "videos_to_merge.txt"
CLOUD_BUCKET_URL=https://storage.cloud.google.com/aivideoscreated
NB_WORDS_PER_SUBTITLE = 15
NB_SECONDS_PER_WORDS = 0.5
DEFAULT_BACKGROUND_MUSIC = medias/royalteefree_background_music.mp3

In [ ]:
!pip install -r requirements.txt

In [ ]:
#@markdown <br><center><img src='https://raw.githubusercontent.com/dropcreations/FFmpeg-for-Google-Colab/32abf44ff4c8d145a94a24611f01141926a8daaa/FFmpeg-Logo.svg' height="40" alt="FFmpeg-logo"/></center>
#@markdown <center><h3><b>Install FFmpeg on Google Colab</b></h3></center><br>
from IPython.display import clear_output
!sudo curl -L https://github.com/BtbN/FFmpeg-Builds/releases/download/latest/ffmpeg-master-latest-linux64-gpl.tar.xz -o /usr/local/bin/ffmpeg.tar.xz
clear_output()
%cd /usr/local/bin/
clear_output()
!7z -y e /usr/local/bin/ffmpeg.tar.xz
clear_output()
!7z -y e /usr/local/bin/ffmpeg.tar
clear_output()
!sudo chmod a+rx /usr/local/bin/ffmpeg
clear_output()
%cd /content/
!sudo curl -L https://mkvtoolnix.download/appimage/MKVToolNix_GUI-70.0.0-x86_64.AppImage -o /usr/local/bin/MKVToolNix_GUI-70.0.0-x86_64.AppImage
!sudo chmod u+rx /usr/local/bin/MKVToolNix_GUI-70.0.0-x86_64.AppImage
!sudo ln -s /usr/local/bin/MKVToolNix_GUI-70.0.0-x86_64.AppImage /usr/local/bin/mkvmerge
!sudo chmod a+rx /usr/local/bin/mkvmerge
clear_output()
!ffmpeg -version

In [ ]:
#Re-going into SDK as ffmpeg install restarts colab
%cd sdk

Importing Vikit.ai Libraries

In [ ]:
from vikit.video.video import Video, VideoBuildSettings
from vikit.video.imported_video import ImportedVideo
from vikit.video.prompt_based_video import PromptBasedVideo
from vikit.video.raw_text_based_video import RawTextBasedVideo
from vikit.video.raw_image_based_video import RawImageBasedVideo
from vikit.video.composite_video import CompositeVideo
from vikit.video.seine_transition import SeineTransition
from vikit.prompt.prompt_factory import PromptFactory
from vikit.common.context_managers import WorkingFolderContext
from vikit.music_building_context import MusicBuildingContext

In [ ]:
from loguru import logger
logger.add("log.txt")

# Vikit.ai usage examples

In [ ]:
#@title Generating a video from simple text prompt
working_folder="./examples/inputs/PromptbasedVideo/"
with WorkingFolderContext(working_folder):
    video_build_settings = VideoBuildSettings(
        music_building_context=MusicBuildingContext(
            apply_background_music=True,
            generate_background_music=True,
        ),
        test_mode=False,
        include_read_aloud_prompt=True,
        target_model_provider="haiper", #Available models: videocrafter, stabilityai, haiper
        output_video_file_name="AICosmetics.mp4",
        interpolate=True,
    )

    prompt = "Unlock your radiance with AI Cosmetics. Experience the magic of premium ingredients, designed to reveal your natural glow. Discover the perfect blend of science and nature with our advanced formulations, tailored to enhance your unique beauty. Transform your skincare routine with our luxurious, high-performance products that deliver visible results. Embrace your true self with confidence, knowing AI Cosmetics has you covered every step of the way."  # @param {type:"string"}
    gw = video_build_settings.get_ml_models_gateway()
    prompt = await PromptFactory(ml_gateway=gw).create_prompt_from_text(prompt)
    video = PromptBasedVideo(prompt=prompt)
    await video.build(build_settings=video_build_settings)

In [ ]:
print(f"Saved image {working_folder + video.media_url}")

from IPython.display import HTML
from base64 import b64encode
mp4 = open(working_folder + video.media_url,'rb').read()
data_url = "data:video/mp4;base64," + b64encode(mp4).decode()
HTML("""
<video width=400 controls>
      <source src="%s" type="video/mp4">
</video>
""" % data_url)

In [ ]:
#@title Generating a more precisely defined video
working_folder="./examples/inputs/CompositeVideo/"
with WorkingFolderContext(working_folder):

    video_build_settings = VideoBuildSettings(
        music_building_context=MusicBuildingContext(
            apply_background_music=True,
            generate_background_music=True,
        ),
        test_mode=False,
        include_read_aloud_prompt=True,
        target_model_provider="haiper",
        output_video_file_name="Output.mp4",
        interpolate=True,
    )

    gw = video_build_settings.get_ml_models_gateway()

    promptText = "Today, we’re journeying back to ancient Greece to explore the life and ideas of ancient philosophers."  # @param {type:"string"}
    prompt = await PromptFactory(ml_gateway=gw).create_prompt_from_text(promptText)

    video_build_settings.prompt = prompt

    vid_cp_final = CompositeVideo()
    vid_cp_final._is_root_video_composite = True

    for sub in prompt.subtitles:
        vid_cp_sub = CompositeVideo()

        keyword_based_vid = RawTextBasedVideo(sub.text)

        prompt_based_vid = RawTextBasedVideo(sub.text)

        vid_cp_sub.append_video(keyword_based_vid).append_video(
            prompt_based_vid
        )
        vid_cp_final.append_video(vid_cp_sub)

    await vid_cp_final.build(build_settings=video_build_settings)
    print(f"Saved video {vid_cp_final.media_url}")

In [ ]:
print(f"Saved image {working_folder + vid_cp_final.media_url}")

from IPython.display import HTML
from base64 import b64encode
mp4 = open(working_folder + vid_cp_final.media_url,'rb').read()
data_url = "data:video/mp4;base64," + b64encode(mp4).decode()
HTML("""
<video width=400 controls>
      <source src="%s" type="video/mp4">
</video>
""" % data_url)

In [ ]:
#@title Generating a video composed of multiple scenes with background music

working_folder="./examples/inputs/MultipleCompositeVideos/"
with WorkingFolderContext(working_folder):
    video_build_settings = VideoBuildSettings(
        music_building_context=MusicBuildingContext(
            apply_background_music=True,
            generate_background_music=True,
        ),
        test_mode=False,
        include_read_aloud_prompt=True,
        target_model_provider="videocrafter",
        output_video_file_name="Coffee.mp4",
        interpolate=True,
    )

    firstScene = "A beautiful and branded coffee looking delicious."  # @param {type:"string"}
    gw = video_build_settings.get_ml_models_gateway()
    prompt = await PromptFactory(ml_gateway=gw).create_prompt_from_text(firstScene)

    vid_cp_final = CompositeVideo()
    vid_cp_final._is_root_video_composite = True

    vid_cp_firstScene = CompositeVideo()

    for sub in prompt.subtitles:
        vid_cp_sub = CompositeVideo()

        keyword_based_vid = RawTextBasedVideo(sub.text)

        prompt_based_vid = RawTextBasedVideo(sub.text)

        transit = SeineTransition(
            source_video=keyword_based_vid,
            target_video=prompt_based_vid,
        )

        vid_cp_sub.append_video(keyword_based_vid).append_video(
            transit
        ).append_video(
            prompt_based_vid
        )
        vid_cp_firstScene.append_video(vid_cp_sub)

    secondScene = "Someone drinking this coffee and enjoying it, this person looks happy"  # @param {type:"string"}

    prompt = await PromptFactory(ml_gateway=gw).create_prompt_from_text(secondScene)
    vid_cp_secondScene = CompositeVideo()

    for sub in prompt.subtitles:
        vid_cp_sub = CompositeVideo()

        keyword_based_vid = RawTextBasedVideo(sub.text)

        prompt_based_vid = RawTextBasedVideo(sub.text)

        transit = SeineTransition(
            source_video=keyword_based_vid,
            target_video=prompt_based_vid,
        )
        vid_cp_sub.append_video(keyword_based_vid).append_video(
            transit
        ).append_video(
            prompt_based_vid
        )
        vid_cp_secondScene.append_video(vid_cp_sub)




    vid_cp_final.append_video(vid_cp_firstScene)
    vid_cp_final.append_video(vid_cp_secondScene)

    await vid_cp_final.build(build_settings=video_build_settings)
    print(f"Saved video {vid_cp_final.media_url}")



In [ ]:
print(f"Saved image {working_folder + vid_cp_final.media_url}")

from IPython.display import HTML
from base64 import b64encode
mp4 = open(working_folder + vid_cp_final.media_url,'rb').read()
data_url = "data:video/mp4;base64," + b64encode(mp4).decode()
HTML("""
<video width=400 controls>
      <source src="%s" type="video/mp4">
</video>
""" % data_url)

In [ ]:
#@title Generating a video from an image

working_folder="./examples/inputs/ImageBasedVideo/"
with WorkingFolderContext(working_folder):
    #Upload image
    from google.colab import files
    image_path = files.upload() #In our example, we used this image: https://storage.needpix.com/rsynced_images/olive-oil-507129_1280.jpg
    #We generate an introduction text video
    video_build_settings_text = VideoBuildSettings(
        test_mode=False,
        include_read_aloud_prompt=False,
        interpolate=True,
        target_model_provider="videocrafter",
    )

    vid_cp_final = CompositeVideo()
    vid_cp_final._is_root_video_composite = True

    text_based_vid_prompt = "The Mediterranean sea and Mediterranean products" #@param {type:"string"}

    text_based_vid = RawTextBasedVideo(text_based_vid_prompt)
    await text_based_vid.prepare_build(build_settings=video_build_settings_text)
    vid_cp_final.append_video(text_based_vid)


    #Then we create an image based video for the last video, for example to showcase a product
    video_build_settings_image = VideoBuildSettings(
        test_mode=False,
        target_model_provider="stabilityai_image",
    )

    image=list(image_path.keys())[0]

    image_prompt = PromptFactory(
            ml_gateway=video_build_settings_image.get_ml_models_gateway()
        ).create_prompt_from_image(
            image_path=image,
            text="Image",
        )

    imageBasedVideo = RawImageBasedVideo(
        raw_image_prompt=image_prompt.image
    )
    imageBasedVideo.build_settings = video_build_settings_image
    video_build_settings_image.prompt = image_prompt

    await imageBasedVideo.prepare_build(build_settings=video_build_settings_image)

    vid_cp_final.append_video(imageBasedVideo)

    video_build_settings = VideoBuildSettings(
        music_building_context=MusicBuildingContext(
            apply_background_music=True,
            generate_background_music=True,
        ),
        test_mode=False,
        output_video_file_name="Image.mp4",
        expected_length=10,
        cascade_build_settings=False,

    )

    await vid_cp_final.build(build_settings=video_build_settings)
    print(f"Saved video {vid_cp_final.media_url}")


In [ ]:
print(f"Saved image {working_folder + vid_cp_final.media_url}")

from IPython.display import HTML
from base64 import b64encode
mp4 = open(working_folder + vid_cp_final.media_url,'rb').read()
data_url = "data:video/mp4;base64," + b64encode(mp4).decode()
HTML("""
<video width=400 controls>
      <source src="%s" type="video/mp4">
</video>
""" % data_url)